In [ ]:
import ixmp
import yaml
import pyam
import nomenclature

In [ ]:
ar6_public = ixmp.Platform("ar6")
ar6_submission = ixmp.Platform("ar6-submission")

In [ ]:
ar6_public_scenario_list = ar6_public.scenario_list()
ar6_submission_scenario_list = ar6_submission.scenario_list()

In [ ]:
with open("region_rename.yaml") as f:
    region_rename_mapping = yaml.safe_load(f)

In [ ]:
eu_climate_dsd = nomenclature.DataStructureDefinition("../definitions/")

In [ ]:
eu_climate = ixmp.Platform("eu-climate-submission")

In [ ]:
project = "2021-SSP"
model = "IMAGE 3.2"
_scenarios = ar6_public_scenario_list[ar6_public_scenario_list.model == model].scenario

_scenarios = [s for s in _scenarios if "SPA" in s]
_scenarios

In [ ]:
_scenarios = [
 'SSP2_SPA0_26I_D',
 'SSP2_SPA1_19I_D_LB',
 'SSP2_SPA1_19I_LIRE_LB',
 'SSP2_SPA1_19I_RE_LB',
 'SSP2_SPA2_19I_D',
 'SSP2_SPA2_19I_LI',
 'SSP2_SPA2_19I_LIRE',
 'SSP2_SPA2_19I_RE',
 'SSP2_SPA2_26I_D',
 'SSP2_SPA2_26I_LI',
 'SSP2_SPA2_26I_LIRE',
 'SSP2_SPA2_26I_RE',
 'SSP2_SPA2_34I_D',
 'SSP2_SPA2_34I_LI',
 'SSP2_SPA2_34I_LIRE',
 'SSP2_SPA2_34I_RE',
 'SSP2_SPA2_45I_D',
 'SSP2_SPA2_45I_LI',
 'SSP2_SPA2_45I_LIRE',
 'SSP2_SPA2_45I_RE']

In [ ]:
scenario_selection = list()
for s in _scenarios:
    scenario_selection.append((model, s))

In [ ]:
scenario_list = [
    pyam.IamDataFrame(
        ixmp.Scenario(mp=ar6_submission, model=m, scenario=s).timeseries()
    )
    for (m, s) in scenario_selection
]
df = pyam.concat(scenario_list)

In [ ]:
df.rename(region=region_rename_mapping, inplace=True)

In [ ]:
image_region_rename_mapping = {
    "IMAGE 3.2|C. Europe": "IMAGE 3.2|Central Europe",
    "IMAGE 3.2|E. Africa": "IMAGE 3.2|Eastern Africa",
    "IMAGE 3.2|N. Africa": "IMAGE 3.2|Northern Africa",
    "IMAGE 3.2|Rest C. America": "IMAGE 3.2|Other Central America",
    "IMAGE 3.2|Rest S. Africa": "IMAGE 3.2|Other Southern Africa",
    "IMAGE 3.2|Rest S. America": "IMAGE 3.2|Other Southern America",
    "IMAGE 3.2|Rest S. Asia": "IMAGE 3.2|Other Southern Asia",
    "IMAGE 3.2|SE. Asia": "IMAGE 3.2|South-East Asia",
    "IMAGE 3.2|W. Africa": "IMAGE 3.2|Western Africa",
    "IMAGE 3.2|W. Europe": "IMAGE 3.2|Western Europe",
}
df.rename(region=image_region_rename_mapping, inplace=True)

In [ ]:
df.region

In [ ]:
df.filter(variable=[
    "Expenditure Share|Food",
    "Disability-Adjusted Life Year|PM2.5",
    "GDP|PPP|Per capita growth rate",
    "GDP|PPP|Per capita ratio to OECD",
    "Premature Deaths|PM2.5",
], keep=False, inplace=True)

In [ ]:
df.rename(unit={
    "bn vkm/yr": "billion vkm/yr",
    "bn pkm/yr": "billion pkm/yr",
    "bn tkm/yr": "billion tkm/yr",
    "bn m2/yr": "billion m2/yr",
    "Mt/year": "Mt/yr",
}, inplace=True)

In [ ]:
df.filter(region=[
    "Brazil", "Canada", "China", "India", 
    "European Union (28 member countries)", "IMAGE 3.2|World",
    "Indonesia", "Japan", "Mexico", "South Africa", 
    "United States of America"], keep=False, inplace=True)

In [ ]:
df.filter(variable=eu_climate_dsd.variable, inplace=True)

In [ ]:
list()

In [ ]:
eu_climate_dsd.validate(df)

In [ ]:
project

In [ ]:
for (_model, _scenario) in df.index:
    _df = df.filter(model=_model, scenario=_scenario)
    # initialize scenario and add timeseries data
    scen = ixmp.Scenario(eu_climate, _model, _scenario, version="new")
    scen.add_timeseries(df.data)
    scen.commit(f"Migrated from the IPCC AR6 database")
    scen.set_as_default()

    scen.set_meta({"Source": f"{project} (via IPCC AR6)"})
    
    print(f"Model {_model} - Scenario {_scenario}" )